# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846811572598                   -0.70    4.2
  2   -7.852286557855       -2.26       -1.53    1.0
  3   -7.852645982290       -3.44       -2.53    3.0
  4   -7.852646677099       -6.16       -3.38    2.5
  5   -7.852646685921       -8.05       -4.76    1.5
  6   -7.852646686725       -9.10       -5.20    4.0
  7   -7.852646686730      -11.32       -5.95    2.0
  8   -7.852646686730      -12.62       -7.62    1.8
  9   -7.852646686730   +    -Inf       -7.60    2.8
 10   -7.852646686730      -14.75       -8.81    2.0
 11   -7.852646686730   +    -Inf      -10.31    2.5
 12   -7.852646686730   +    -Inf      -10.29    1.8
 13   -7.852646686730   +    -Inf      -11.58    2.0
 14   -7.852646686730   +    -Inf      -12.33    3.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846780943025                   -0.70           4.5
  2   -7.852523356273       -2.24       -1.64   0.80    2.2
  3   -7.852635324646       -3.95       -2.72   0.80    1.0
  4   -7.852646482072       -4.95       -3.22   0.80    2.0
  5   -7.852646672172       -6.72       -4.05   0.80    1.5
  6   -7.852646686387       -7.85       -4.78   0.80    1.5
  7   -7.852646686718       -9.48       -5.62   0.80    2.0
  8   -7.852646686730      -10.94       -7.01   0.80    2.0
  9   -7.852646686730      -12.71       -7.27   0.80    3.0
 10   -7.852646686730   +  -14.75       -8.04   0.80    1.0
 11   -7.852646686730      -14.27       -8.46   0.80    2.0
 12   -7.852646686730   +  -14.45       -9.13   0.80    1.2
 13   -7.852646686730   +    -Inf       -9.58   0.80    2.2
 14   -7.852646686730   +  -14.75      -10.97   0.80    1.2
 15   -7.852646686730      -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.412976e+01     3.587663e+00
 * time: 0.39490294456481934
     1     1.085646e+00     1.968111e+00
 * time: 0.5905249118804932
     2    -1.737782e+00     1.943858e+00
 * time: 0.6136119365692139
     3    -3.704627e+00     1.766677e+00
 * time: 0.6465868949890137
     4    -5.113320e+00     1.519755e+00
 * time: 0.6796128749847412
     5    -6.622301e+00     1.145451e+00
 * time: 0.712993860244751
     6    -7.352095e+00     6.554711e-01
 * time: 0.746006965637207
     7    -7.617213e+00     3.631565e-01
 * time: 0.7689058780670166
     8    -7.720524e+00     1.281307e-01
 * time: 0.7921009063720703
     9    -7.765364e+00     2.214731e-01
 * time: 0.8152530193328857
    10    -7.801816e+00     7.761197e-02
 * time: 0.8385410308837891
    11    -7.828809e+00     6.880180e-02
 * time: 0.8614909648895264
    12    -7.841190e+00     5.079711e-02
 * time: 0.8849289417266846
    13    -7.845306e+00     5.990287e-02
 * time: 0.90793991088

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846803339538                   -0.70    5.0
  2   -7.852320332668       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686712                   -2.54
  2   -7.852646686730      -10.74       -5.97
  3   -7.852646686730      -15.05      -12.70


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.414926473789687e-13
|ρ_newton - ρ_scfv| = 3.1321987486148424e-13
|ρ_newton - ρ_dm|   = 2.3898025591069963e-10
